# Other examples
A notebook to test simple layers

In [1]:
%cd ../..

/home/karimgamaleldin/projects/KTorch


In [2]:
# imports
import numpy as np
from nn import Flatten, Linear, Dropout, Sequential, ReLU
from core import KTorch
from autograd import Tensor

### 1. Flatten

In [3]:
# create a tensor
np.random.seed(0)
x = np.random.randn(6, 128, 128, 3).astype(np.float32)
x_tensor = Tensor(x)
flatten = Flatten()
flatten(x_tensor).shape, 128*128*3

((6, 49152), 49152)

In [4]:
# Compare output
flattened = flatten(x_tensor)
np_flattened = x.reshape(6, -1)
np.equal(flattened.data, np_flattened).all()

True

#### Test backpropagation

In [5]:
# Create input
np.random.seed(0)
x = np.random.randn(6, 128, 128, 3).astype(np.float32)
x_tensor = Tensor(x)

In [6]:
# Create model
flatten = Flatten()
linear = Linear(128*128*3, 10)

# Forward pass
flattened = flatten(x_tensor)
output = linear(flattened)
output.shape

(6, 10)

In [7]:
# Manual forward pass

# Parameters
w = linear.weight.data
b = linear.bias.data

np_flattened = x.reshape(6, -1)
manual_output = np_flattened @ w + b
np.equal(output.data, manual_output).all()

True

In [8]:
# Backward pass
output.backward()

In [9]:
# Manual backward pass
grad_output = np.ones_like(output.data)
grad_flattened = grad_output @ w.T
grad_flattened = grad_flattened.reshape(6, 128, 128, 3)
grad_x = grad_flattened
w_grad = np_flattened.T @ grad_output
b_grad = grad_output.sum(axis=0)

np.equal(x_tensor.grad.data, grad_x).all(), np.equal(linear.weight.grad.data, w_grad).all(), np.equal(linear.bias.grad.data, b_grad).all()

(True, True, True)

### 2. Dropout

In [10]:
# Create tensor
np.random.seed(0)
x = np.random.randn(32, 128).astype(np.float32)
x_tensor = Tensor(x)

In [11]:
# Dropout
dropout = Dropout(0.5)
linear = Linear(128, 10)

output, mask = dropout(x_tensor)
output = linear(output)

output.shape

(32, 10)

In [12]:
# Manual forward propagation
mask_np = mask.data
w = linear.weight.data
b = linear.bias.data


masked = x * mask_np * (1.0 / 0.5)
output_np = masked @ w + b

In [13]:
output_np.shape, np.equal(output.data, output_np).all()

((32, 10), True)

In [14]:
# Backward pass
output.backward()

In [15]:
# Manual backward pass
grad_output = np.ones_like(output.data)
grad_masked = grad_output @ w.T
grad_x = grad_masked * mask_np * (1.0 / 0.5)
w_grad = masked.T @ grad_output
b_grad = grad_output.sum(axis=0)

np.equal(x_tensor.grad.data, grad_x).all(), np.equal(linear.weight.grad, w_grad).all(), np.equal(linear.bias.grad, b_grad).all()

(True, True, True)

### Sequential

In [18]:
# Get input
np.random.seed(0)
x = np.random.randn(32, 128).astype(np.float32)
x_tensor = Tensor(x)

# Create layers
linear_1 = Linear(128, 64)
relu = ReLU()
linear_2 = Linear(64, 10)

# Forward pass
output = linear_1(x_tensor)
output = relu(output)
output = linear_2(output)

output.shape

(32, 10)

In [21]:
output.backward()

In [22]:
# Grad values
linear_2_weight_grad = linear_2.weight.grad
linear_2_bias_grad = linear_2.bias.grad
linear_1_weight_grad = linear_1.weight.grad
linear_1_bias_grad = linear_1.bias.grad

In [24]:
linear_2.weight.zero_grad()
linear_2.bias.zero_grad()
linear_1.weight.zero_grad()
linear_1.bias.zero_grad()

# Sequential forward pass
seq = Sequential(linear_1, relu, linear_2)
output_seq = seq(x_tensor)
output_seq.shape

afdafasa
afdafasa


(32, 10)

In [26]:
np.equal(output.data, output_seq.data).all()

True

In [27]:
output_seq.backward()

# Grad values
np.equal(linear_2_weight_grad, linear_2.weight.grad).all(), np.equal(linear_2_bias_grad, linear_2.bias.grad).all(), np.equal(linear_1_weight_grad, linear_1.weight.grad).all(), np.equal(linear_1_bias_grad, linear_1.bias.grad).all()

(True, True, True, True)